# RAG - Pioneers
![](https://python.langchain.com/assets/images/rag_concepts-4499b260d1053838a3e361fb54f376ec.png)

Together ai-аас хэлний загвар ашигласан.

In [1]:
!pip install -qU langchain-openai

In [2]:
%%capture
!pip install openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet

In [ ]:
import os
os.kill(os.getpid(), 9)

In [1]:
%pip install langgraph


Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_community.document_loaders import TextLoader

## Vector Store үүсгэлт
RAG систем сайн хариу өгөхийн тулд хоёр зүйл маш чухал байгаа юм. 

Нэг: Асуултанд харгалзах зөв өгөгдлөө ретрив хийсэн эсэх

Хоёр : LLM загвар промптоор ирсэн өгөгдөл болон асуултыг ашиглан зөв хариулт боловсруулж байгаа эсэх (холбогдох зөв өгөгдөлд төвлөрч чадаж байгаа эсэх)

Тиймээс бүх өгөгдлөө ганц вектор санд хадгалж түүнээс хайлт хийх нь буруу өгөгдлийг ретрив хийж авах магадлалыг нэмэгдүүлж байна. 

 Өөрөөр хэлбэл бид өгөгдлүүдээ ангилж өөр өөр вектор сан үүсгээд хадгалсан. Жишээ нь: салбарын мэдээлэлтэй холбоотой өгөгдлүүдээ өөр тусад нь хадгалах гэх мэт
![](https://miro.medium.com/v2/resize:fit:1100/format:webp/1*vAvDBIbr8MnL_Q51mBtBhw.png)

### Өгөгдөл унших

`data` хавтсанд байрлах `csv` файлыг уншиж LangChain [Document](https://python.langchain.com/api_reference/core/documents/langchain_core.documents.base.Document.html)-н жагсаалт болгоно.

- `khas_bank_news.csv`
- `khas_bank_pages.csv`
- `khas_bank_products.csv`

### Нэмж салбарын мэдээлэл орууллаа.

In [80]:
DATA_DIR = "../data/"
loader = CSVLoader(file_path=f"{DATA_DIR}khas_bank_news.csv",
                   source_column="link",
                   metadata_columns=["title", "date", "link"],
                   content_columns=["content"],
                   csv_args={
                       "fieldnames": ["title", "link", "date", "content"],
                       }
                   )
news_data = loader.load()[1:]

loader = CSVLoader(file_path=f"{DATA_DIR}khas_bank_products_2.csv",
                   source_column="link",
                   metadata_columns=["id", "side_menu_text", "link"],
                   content_columns=["content"],
                   csv_args={
                       "fieldnames": ["id", "content", "side_menu_text", "link"],
                       }
                   )
product_data = loader.load()[1:]

loader = CSVLoader(file_path=f"{DATA_DIR}khas_bank_pages_zasvar.csv",
                   source_column="link",
                   metadata_columns=["title", "link"],
                   content_columns=["content"],
                   csv_args={
                       "fieldnames": ["title", "content", "link"],
                       }
                   )
pages_data = loader.load()[1:]

from langchain_community.document_loaders import TextLoader
loader = TextLoader(file_path=f"{DATA_DIR}branch.txt")
branch_data = loader.load()

# loader.load()
all_data = pages_data + product_data + news_data # + branch_data
print(len(news_data), len(product_data), len(pages_data))
print(f"Нийт хуудасны тоо: {len(all_data)}")
print(all_data[0])

664 71 50
Нийт хуудасны тоо: 785
page_content='content: бидний тухай
хасбанк 2001 онд байгууллагдсан цагаасаа монгол улсын банк санхүүгийн системд тогтвортой өсөлтийг бий болгож, үйл ажиллагаагаа өргөтгөж ирсэн бөгөөд өдгөө банкны системд нөлөө бүхий 5 банкны нэг болоод байна.
банкны хэтийн зорилт болон үнэт зүйлс
бидний хэтийн зорилт
•
бид монголдоо дэлхийн түвшний санхүүгийн үйлчилгээтэй байгууллага болно.
бидний үнэт зүйлс
хүртээмжтэй
санаачилгатай
нийгмийн хариуцлагатай
харилцагч төвтэй
эрч хүчтэй
мэргэжлийн
бидний түүх
компанийн засаглал, нийгмийн хариуцлагын хүрээнд авч хэрэгжүүлж буй арга хэмжээгээрээ салбартаа жишиг тогтоож буй хасбанк нь эх дэлхий, эрдэнэт хүмүүн, эрүүл ашгийг эрхэмлэсэн, санхүүгийн зах зээлд манлайлах үйлчилгээтэй тэргүүлэгч банк байх эрхэм зорилгыг өмнөө тавин монгол орон даяар үйл ажиллагаа явуулж буй 71 салбар нэгж, бизнес үйлчилгээний төвүүд, тэдгээрт ажиллаж буй 1,152 мэргэшсэн ажилтан, 8,000 гаруй мерчант, 360 гаруй атм-аар дамжуулан 1 сая харилцагчдада

In [93]:
# from langchain_community.document_loaders import TextLoader
# loader = TextLoader(file_path=f"{DATA_DIR}branch.txt")
# loader.load()

In [62]:
# branch_data = loader.load()

### Өгөгдөл хуваах
Өгөгдлүүдээ хамааралтай хэсгүүдээр нь хуваасан.

Салбарын мэдээллийг 300 Chunk size-тай гэх мэтээр

### chunk size
Хэрэглэх гэж буй эмбэддинг загварынхаа оролтын max_sequence уртаас хамаарч chunk хэмжээгээ сонгох нь хамгийн зөв сонголт болох юм. Учир нь chunk хэмжээ нь загварынхаа оролтын уртаас хэтрэсэн тохиолдолд эмбэддинг загварууд оролтын дарааллыг зөвшөөрөгдсөн хамгийн их дарааллын урт хүртэл багасгадаг. Энэ нь хязгаараас хэтэрсэн токеныг үл тоомсорлох болно гэсэн үг.

Энэ нь текстийн чухал хэсгүүдийг тасласан тохиолдолд чухал мэдээлэл алдагдах юм.

In [81]:
from langchain.schema import Document

all_data = [
    Document(page_content=doc.page_content.lower(), metadata=doc.metadata)
    for doc in all_data
]


In [82]:
product_data = [
    Document(page_content=doc.page_content.lower(), metadata=doc.metadata)
    for doc in product_data
]

In [83]:
branch_data = [
    Document(page_content=doc.page_content.lower(), metadata=doc.metadata)
    for doc in branch_data
]


In [84]:
MARKDOWN_SEPARATORS = [
    "\n\n",
    "",
]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # The maximum number of characters in a chunk: we selected this value arbitrarily
    chunk_overlap=100,  # The number of characters to overlap between chunks
    # add_start_index=True,  # If `True`, includes chunk's start index in metadata
    strip_whitespace=True,  # If `True`, strips whitespace from the start and end of every document
    separators=MARKDOWN_SEPARATORS,
)

docs = text_splitter.split_documents(documents=all_data)

In [85]:
#chunk_size = 500
MARKDOWN_SEPARATORS = [
    "\n\n",
    "",
]

text_splitter_product = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # The maximum number of characters in a chunk: we selected this value arbitrarily
    chunk_overlap=200,  # The number of characters to overlap between chunks
    # add_start_index=True,  # If `True`, includes chunk's start index in metadata
    strip_whitespace=True,  # If `True`, strips whitespace from the start and end of every document
    separators=MARKDOWN_SEPARATORS,
)

product_docs = text_splitter.split_documents(documents=product_data)

In [86]:
text_splitter_branch = RecursiveCharacterTextSplitter(
    chunk_size=300, chunk_overlap=100,
    separators = [
        '\n\n',
        "\n"
    ]
)
branch_docs = text_splitter_branch.split_documents(documents=branch_data)

In [87]:
docs[50]

Document(metadata={'source': 'https://www.xacbank.mn/page/award', 'row': 3, 'title': 'шагнал урамшуулал', 'link': 'https://www.xacbank.mn/page/award'}, page_content='гжилд бодитой хувь нэмрээ оруулж буй шилдэг аж ахуйн нэгж, бизнес эрхлэгчдийг шалгаруулан өргөмжилдөг “entrepreneur” ёслолын ажиллагаанаас банкны салбарын 2013 оны онцлох энтрепренерээр шалгаран “хүрэл меркури” цом хүртлээ.\nнийслэлийн 374 жилийн ойн өдрийг тохиолдуулан нитх, монголын сурагчдын холбоо хамтран нийслэлийн хүүхдийн сайн сайхны төлөө хийсэн бодитой ажил, санал санаачлагыг үнэлж сурагчдын дунд явуулсан судалгааг үндэслэж “нийслэлийн хүүхдэд ээлтэй шилдэг байгууллага”-аар хасбанкийг ша')

In [12]:
# # sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
from sentence_transformers import SentenceTransformer

### Embedding

Embedding нь энгийн үгээр ямар нэгэн өгөгдлийг вектор дүрслэл рүү хөрвүүлсэн хэлбэр бөгөөд тухайн вектор нь олон хэмжээст огторгуйд ижил, төстэй утга бүхий өгөгдөлтэй ойр, эсрэг утгатай өгөгдлөөс хол байрладаг.

Энэхүү жишээ RAG-д [`sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2`](https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2) загварыг сонгон авлаа.

![embedding model](https://python.langchain.com/assets/images/embeddings_concept-975a9aaba52de05b457a1aeff9a7393a.png)

In [88]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
# embeddings = HuggingFaceEmbeddings(model_name="bayartsogt/mongolian-roberta-base")
# embeddings = HuggingFaceEmbeddings(model_name="distiluse-base-multilingual-cased-v2")

In [ ]:
!pip install sentence_transformers

### Vector Store

Харин vector store нь embedding болон өгөгдөл эсвэл түүний заагчийг хамтад нь хадгалж, хайлтын query орж ирэхэд мөн түүнийг вектор хэлбэрт хөрвүүлэн, санд байх векторуудаас хамгийн ойрыг хайдаг өгөгдлийн сан юм.

Энэхүү жишээнд FAISS vector store сонгож ашиглалаа.

![vectorestore](https://python.langchain.com/assets/images/vectorstores-2540b4bc355b966c99b0f02cfdddb273.png)

In [89]:
# Бүх document-с embedding үүсгэж, vector store-д хадгалах
# (өмнө нь үүсгэсэн бол заавал дахин үүсгэх шаардлагагүй, доорх load_local функцээр хадгалсан файлаас уншиж болно)
vector_store = FAISS.from_documents(docs, embeddings)
vector_store.save_local("faiss_index")
branch_store = FAISS.from_documents(branch_docs, embeddings)
branch_store.save_local("faiss_branch")
product_store = FAISS.from_documents(product_docs, embeddings)
product_store.save_local("faiss_product")

In [90]:
# Үүсгэсэн vector store-г файлаас унших
vector_store = FAISS.load_local(
    "faiss_index", embeddings, allow_dangerous_deserialization=True
)
branch_store = FAISS.load_local(
    "faiss_branch", embeddings, allow_dangerous_deserialization=True
)
product_store = FAISS.load_local( 
    "faiss_product", embeddings, allow_dangerous_deserialization=True
)

In [91]:
# vector store-с document хайгч, k параметраар хамгийн ойр утгатай хэдэн document буцаахыг дамжуулна
retriever_all_data = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})
retriever_branch_data = branch_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})
retriever_product_data = product_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})
# docs = retriever.invoke("хасбанк хэзээ IPO хийсэн бэ")
# docs = retriever.invoke("хэрхэн тэтгэлэгт хамрагдах вэ")
retriever_branch_data.invoke("сүхбаатар дүүргийн салбарын байршил хаана байдаг вэ")

[Document(metadata={'source': '../data/branch.txt'}, page_content='сүхбаатар салбар нь сүхбаатар дүүрэг, 7-р хороо, эрхүүгийн гудамж, ориент төвийн 1-р давхарт байрладаг. даваа - баасан: 8:30-16:30 цагийн хооронд ажиллах ба бямба, ням: амарна, холбогдох дугаар 1800-1888.'),
 Document(metadata={'source': '../data/branch.txt'}, page_content='төв салбар нь сүхбаатар дүүрэг, энхтайваны өргөн чөлөө, чингүнжавын гудамж-30-д байрладаг. даваа - баасан: 8:30-16:30 цагийн хооронд ажиллах ба бямба, ням: амарна, холбогдох дугаар 1800-1888.'),
 Document(metadata={'source': '../data/branch.txt'}, page_content='төв талбай салбар нь сүхбаатар дүүрэг, 8-р хороо, ерөнхий сайд амарын гудамж, хасбанкны төв байр байрладаг. даваа - баасан: 8:30-16:30 цагийн хооронд ажиллах ба бямба, ням: амарна, холбогдох дугаар 1800-1888.'),
 Document(metadata={'source': '../data/branch.txt'}, page_content='цогтцэций тооцооны төв нь цогтцэций сум, цэций баг, хасбанкны байрлалд байрладаг. даваа - баасан: 8:30-16:30 цагийн х

In [92]:
retriever_product_data.invoke("Харилцах дансны жилийн хүү хэд вэ?")

[Document(metadata={'source': 'https://www.xacbank.mn/product/36', 'row': 16, 'id': '36', 'side_menu_text': 'ирээдүйн саятан /хүүхдийн хадгаламж/\r\nдансны доод үлдэгдэл 20,000₮\r\nзарласан хүү 11.0%\r\nбодит хүү 11.57%', 'link': 'https://www.xacbank.mn/product/36'}, page_content='өхцөл\r\nвалютын төрөл\r\nдоод үлдэгдэл\r\nхугацаа\r\nжилийн хүү\r\nзарласан хүү\r\nбодит хүү\r\nгэрээ цуцлах хүү\r\nтөгрөг\r\n20,000 төгрөг\r\nхүүхдийн 18 насны төрсөн өдөр хүртэл\r\n11.0%\r\n11.57%\r\n2.40%\r\nданс нээхэд\r\nхураамжгүй\r\nгэрээний хугацаанаас өмнө гэрээ цуцлах хураамж\r\n3,000₮\r\nбусад шимтгэл, хураамж\r\nүйлчилгээний төрөл\r\nшимтгэлийн хувь хэмжээ\r\n(мөнгөн тэмдэгтийн төрөл)\r\nmnt\r\nорлогын гүйлгээ\r\nбэлэн бус орлого\r\nшимтгэлгүй\r\nбэлэн орлого\r\nшимтгэлгүй\r\nтодорхойлолт *\r\n5,000₮\r\nхуулга\r\nцахим шуудангаар илгээхэд\r\nшим'),
 Document(metadata={'source': 'https://www.xacbank.mn/product/71', 'row': 38, 'id': '71', 'side_menu_text': 'хугацаатай хадгаламж\r\nзарласан хүү (төг

Vector Store үүсгэсэн бол түүнээс ойролцоо утгатай өгөгдөл хайж үзье.

![retriever](https://python.langchain.com/assets/images/retriever_concept-1093f15a8f63ddb90bd23decbd249ea5.png)

In [93]:
# vector store-с document хайгч, k параметраар хамгийн ойр утгатай хэдэн document буцаахыг дамжуулна
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})

# docs = retriever.invoke("хасбанк хэзээ IPO хийсэн бэ")
# docs = retriever.invoke("хэрхэн тэтгэлэгт хамрагдах вэ")
retriever.invoke("хэрхэн тэтгэлэгт хамрагдах вэ")

[Document(metadata={'source': 'https://www.xacbank.mn/article/514', 'row': 371, 'title': 'гүйцэтгэх захирлын илгээлт', 'date': '2015.05.11', 'link': 'https://www.xacbank.mn/article/514'}, page_content='гч байгууллагуудад туслалцаа үзүүлэв.бид санхүүгийн боловсролыг нийгэм, олон нийтэд түгээх замаар банкны бүтээгдэхүүн, үйлчилгээгээ таниулж ирсэн хандивын хөтөлбөрүүдээ үргэлжлүүллээ. өнөөдрийн байдлаар улаанбаатар, хөдөө орон нутгийн 38,140 хүүхэд, багш болон сайн дурын ажилтнууд хөтөлбөрт хамрагдан ач тусыг нь хүртээд байна.банкны зээлийн бодлогоор харилцагчийн үнэлгээг нийгэм, байгаль орчны чиглэлийн хууль тогтоомж, шаардлагуудад нийцэж байгаа эсэхээс хамааруулан тодорхойлж, хяналт мониторинг'),
 Document(metadata={'source': 'https://www.xacbank.mn/article/477', 'row': 409, 'title': 'таны хүүхдийн баталгаат ирээдүй – ирээдүйн саятан хадгаламж', 'date': '2014.05.29', 'link': 'https://www.xacbank.mn/article/477'}, page_content='охион байгуулагдах ирээдүйн саятан хадгаламж эзэмшигчдэд зо

## Өгөгдөл хайх, хариулах

Өмнөх хэсэгт өгөгдлөө бэлтгэж, vector store-т хадгалсан бол энэ хэсэгт хэрэглэгчийн асуусан асуултаар vector store-с хайлт хийж, LLM-р хариулт өгөх болно.

![](https://miro.medium.com/v2/resize:fit:2000/format:webp/1*8dUBh25peTpsucMpL1ypUg.png)

### LLM

Хайж олсон мэдээлэл, хэрэглэгчийн асуултад тохирох хариулт үүсгэхийн тулд бичвэр үүсгэгч Large Language Model ашиглана.

Энд Meta компанийн сургасан [8 тэрбум параметртай Llama 3.1(https://huggingface.co/meta-llama/Llama-3.1-8B)] загварыг сонгов. (Hugging Face дээр загвар ашиглах зөвшөөрөл хүссэн маягт бөглөх ёстой)


In [84]:
# Hugging Face-с хандах эрх авсан хаалттай загваруудад хандахын тулд нэвтэрсэн байх шаардлагатай
# Нэмэлт мэдээллийг https://huggingface.co/docs/huggingface_hub/package_reference/authentication
from huggingface_hub import notebook_login

notebook_login()

In [94]:
import os
if not os.environ.get("HF_TOKEN"):
    os.environ["HF_TOKEN"]= ""
if not os.environ.get("TOGETHER_API_KEY"):
    os.environ["TOGETHER_API_KEY"] = ""
os.environ["LANGCHAIN_TRACING_V2"] = "true"
if not os.environ.get("LANGCHAIN_API_KEY"):
    os.environ["LANGCHAIN_API_KEY"] = ""

In [95]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    base_url="https://api.together.xyz/v1",
    api_key=os.environ["TOGETHER_API_KEY"],
    model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
)

### Chain: Retrieval -> Prompt -> LLM

Prompt-н `{context}`-н оронд хайж олсон мэдээлэл, `{input}`-н оронд хэрэглэгчийн асуусан асуулт орох болно.

In [96]:
import pandas as pd

keywords_df = pd.read_csv("../data/classifier.csv")

keyword_dict = {}
for _, row in keywords_df.iterrows():
    keyword = row['text'].lower().strip()
    label = row['label'].strip()

    if label not in keyword_dict:
        keyword_dict[label] = set()
    keyword_dict[label].add(keyword)


In [97]:
def classify_query(query: str):
    query_lower = query.lower()

    for label, keywords in keyword_dict.items():
        if any(keyword in query_lower for keyword in keywords):
            return label

    return "General_Request"


In [98]:
from langchain_core.tools import tool

@tool(response_format="content_and_artifact")
def retrieve(query: str):
    """Classify the query using keyword matching."""
    classification = classify_query(query)

    if classification == "branch_Request":
        retrieved_docs = branch_store.similarity_search(query, k=5)
    elif classification == "product_Request":
        retrieved_docs = product_store.similarity_search(query, k=5)
    else:
        retrieved_docs = vector_store.similarity_search(query, k=5)

    # Serialize the results
    serialized = "\n\n".join(
        f"Source: {doc.metadata}\nContent: {doc.page_content}"
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs


In [ ]:
# from langchain_core.tools import tool
# @tool(response_format="content_and_artifact")
# def retrieve(query: str):
#     """choose which store will use"""
#     if "хаана байрладаг вэ" in query or "байршил хаана байдаг вэ" in query:
#         retrieved_docs = branch_store.similarity_search(query, k=5)
#         # retrieved_docs = vector_store.similarity_search(query, k=5)
#         serialized = "\n\n".join(
#             (f"Source: {doc.metadata}\n" f"Content: {doc.page_content}")
#             for doc in retrieved_docs
#         )
#         return serialized, retrieved_docs
#     else :
#         retrieved_docs = vector_store.similarity_search(query, k=5)
#         serialized = "\n\n".join(
#             (f"Source: {doc.metadata}\n" f"Content: {doc.page_content}")
#             for doc in retrieved_docs
#         )
#         return serialized, retrieved_docs

In [99]:
from langgraph.graph import MessagesState, StateGraph

graph_builder = StateGraph(MessagesState)

In [100]:
from langchain_core.messages import SystemMessage
from langgraph.prebuilt import ToolNode
# Step 1: Generate an AIMessage that may include a tool-call to be sent.
def query_or_respond(state: MessagesState):
    """Generate tool call for retrieval or respond."""
    llm_with_tools = llm.bind_tools([retrieve])
    # return state['messages']
    response = llm_with_tools.invoke(state["messages"])
    # MessagesState appends messages to state instead of overwriting
    return {"messages": [response]}


# Step 2: Execute the retrieval.
tools = ToolNode([retrieve])


# Step 3: Generate a response using the retrieved content.
def generate(state: MessagesState):
    """Generate answer."""
    # Get generated ToolMessages
    recent_tool_messages = []
    for message in reversed(state["messages"]):
        if message.type == "tool":
            recent_tool_messages.append(message)
        else:
            break
    tool_messages = recent_tool_messages[::-1]

    # Format into prompt
    docs_content = "\n\n".join(doc.content for doc in tool_messages)
    system_message_content = (
        "You are an mongolian assistant for question-answering tasks. "
        "Use the following pieces of retrieved context to answer "
        "the question. If you don't know the answer, say that you "
        "don't know. Use three sentences maximum and keep the "
        "answer concise, and use mongolian."
        "\n\n"
        f"{docs_content}"
    )
    conversation_messages = [
        message
        for message in state["messages"]
        if message.type in ("human", "system")
        or (message.type == "ai" and not message.tool_calls)
    ]
    prompt = [SystemMessage(system_message_content)] + conversation_messages

    # Run
    response = llm.invoke(prompt)
    return {"messages": [response]}

In [101]:
from langgraph.graph import END
from langgraph.prebuilt import ToolNode, tools_condition

graph_builder.add_node(query_or_respond)
graph_builder.add_node(tools)
graph_builder.add_node(generate)

graph_builder.set_entry_point("query_or_respond")
graph_builder.add_conditional_edges(
    "query_or_respond",
    tools_condition,
    {END: END, "tools": "tools"},
)
graph_builder.add_edge("tools", "generate")
graph_builder.add_edge("generate", END)

graph = graph_builder.compile()

In [102]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
# graph = graph_builder.compile(checkpointer=memory)

# Specify an ID for the thread
config = {"configurable": {"thread_id": "abc123"}}


In [48]:
input_message = "төв салбар хаана байрладаг вэ"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

төв салбар хаана байрладаг вэ
================================== Ai Message ==================================
Tool Calls:
  retrieve (call_cpccobumy2z3cp0gj1elg8a0)
 Call ID: call_cpccobumy2z3cp0gj1elg8a0
  Args:
    query: төв салбар хаана байрладаг вэ
================================= Tool Message =================================
Name: retrieve

Source: {'source': '../data/branch.txt'}
Content: налайх салбар нь налайх дүүрэг, 2-р хороо, дамдины гудамж, хасбанкны байр. даваа - баасан: 8:30-16:30 цагийн хооронд ажиллах ба бямба, ням: амарна, холбогдох дугаар 1800-1888.

Source: {'source': '../data/branch.txt'}
Content: гранд тооцооны төв нь баянгол дүүрэг, 2-р хороо, энхтайваны өргөн чөлөө-46, "гранд плаза" худалдааны төв, b1 давхарт байрладаг. даваа - баасан: 08:30-16:30 цагийн хооронд ажиллах ба бямба, ням: амарна, холбогдох дугаар 1800-1888.

Source: {'source': '../data/branch.txt'}
Content: нарантуу

In [49]:
input_message = "Цалингийн зээл нэг сарын хүү хэд вэ?"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Цалингийн зээл нэг сарын хүү хэд вэ?
================================== Ai Message ==================================
Tool Calls:
  retrieve (call_o19ag1pn7egzxkpyasm8zama)
 Call ID: call_o19ag1pn7egzxkpyasm8zama
  Args:
    query: Цалингийн зээл нэг сарын хүү
================================= Tool Message =================================
Name: retrieve

Source: {'source': 'https://www.xacbank.mn/product/1', 'row': 1, 'id': '1', 'side_menu_text': 'цалингийн зээл\r\nхэмжээ\r\nзээлжих чадвараар тодорхойлогдоно\r\nхугацаа 30 сар\r\nхүү:\r\nсарын: 1.5% - 1.9%\r\nжилийн: 18.0% - 22.8%', 'link': 'https://www.xacbank.mn/product/1'}
Content: үндсэн төлбөр тэнцүү эсхүл сард төлөх нийт төлбөр тэнцүү сонголттой.
барьцаа хөрөнгө: таны цалингийн орлого. шаардлагатай тохиолдолд нэмэлт барьцаа хөрөнгө.
тавигдах шаардлага: хуулийн этгээдэд 6 сараас доошгүй хугацаанд ажилласан, үндсэн ажилтан байх; зээлийн муу түүхгүй ба

In [104]:
input_message = "Хас банкны IPO хэзээ зарлагдсан вэ?"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Хас банкны IPO хэзээ зарлагдсан вэ?
================================== Ai Message ==================================
Tool Calls:
  retrieve (call_ilm9hzu90kobqkuclgupqskd)
 Call ID: call_ilm9hzu90kobqkuclgupqskd
  Args:
    query: Хас банкны IPO хэзээ зарлагдсан
================================= Tool Message =================================
Name: retrieve

Source: {'source': 'https://www.xacbank.mn/article/ipo-opening', 'row': 73, 'title': 'хасбанкны ipo: нээлтийн арга хэмжээ амжилттай боллоо', 'date': '2023.05.25', 'link': 'https://www.xacbank.mn/article/ipo-opening'}
Content: захиалгыг хасбанкны дижитал банк болон үндсэн андеррайтер райнос инвестмент үцк, туслах андеррайтер өлзий энд ко капитал үцк-уудаар өгч болохоос гадна 5-р сарын 29-нд анхдагч арилжаа нээгдмэгц өөрийн бүртгэлтэй аль ч үнэт цаасны компаниар дамжуулан захиалга өгөх боломжтой.мөн хасбанкны ipo-ын талаарх бүхий л мэдээллийг ipo.xacbank